# **Finetuning del modelo T5**

En este notebook llevamos a cabo el finetuning del modelo T5. Se trata de un proceso complejo, que hubo que corregir varias veces, debido a la incompatibilidad de librerías, en especial con accelerate, y problemas con la tokenización.

Comenzamos con la instalación de las librerías, que ya digo que dieron problemas.

In [ ]:
"""
FINE-TUNING T5 PARA ANÁLISIS ELECTORAL ESPAÑOL
Google Colab Pro - VERSIÓN ESTABLE Y CORREGIDA
Con solución integrada para error "piece id is out of range"
"""

# ============================================================================
# CELDA 1: INSTALACIÓN DE DEPENDENCIAS (VERSIÓN ESTABLE)
# ============================================================================
print("🚀 INICIANDO FINE-TUNING T5 - VERSIÓN ESTABLE")
print("=" * 70)

# Instalación de dependencias COMPATIBLES
!pip install -q transformers==4.36.0
!pip install -q accelerate==0.21.0  # Specify compatible accelerate version
!pip install -q datasets==2.15.0
!pip install -q evaluate==0.4.0
!pip install -q rouge-score==0.1.2
!pip install -q nltk==3.8.1
!pip install -q sentencepiece==0.1.99
!pip install -q protobuf==3.20.3
!pip install -q peft==0.5.0

import os
import sys
import json
import torch
import numpy as np
import pandas as pd
from datetime import datetime
import gc
import warnings
warnings.filterwarnings('ignore')

print("✅ Dependencias instaladas")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

🚀 INICIANDO FINE-TUNING T5 - VERSIÓN ESTABLE
✅ Dependencias instaladas
PyTorch: 2.9.0+cu126
CUDA disponible: True


DeepSeek incluyó esta celda de revisión de recursos después de que le dijera que iba a usar la GPU L4 de Colab Pro.

In [ ]:
# ============================================================================
# CELDA 2: VERIFICACIÓN DE RECURSOS Y CONFIGURACIÓN
# ============================================================================
def check_resources_and_setup():
    """Verifica recursos y configura parámetros seguros"""

    print("\n" + "=" * 70)
    print("🔍 VERIFICACIÓN DE RECURSOS - Google Colab Pro")
    print("=" * 70)

    # Información de GPU
    gpu_info = {}
    if torch.cuda.is_available():
        gpu_info['name'] = torch.cuda.get_device_name(0)
        gpu_info['memory_gb'] = torch.cuda.get_device_properties(0).total_memory / 1e9
        gpu_info['available_memory'] = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()

        print(f"🎯 GPU: {gpu_info['name']}")
        print(f"💾 Memoria GPU total: {gpu_info['memory_gb']:.2f} GB")
        print(f"💾 Memoria GPU disponible: {gpu_info['available_memory'] / 1e9:.2f} GB")

        # Seleccionar modelo según GPU
        if gpu_info['memory_gb'] >= 16:  # V100/A100
            model_name = "google-t5/t5-base"  # 220M parámetros
            batch_size = 4  # Reducido para ser conservador
            print("   → Usando T5-base (220M parámetros) con batch size 4")
        else:  # T4 15GB o menos
            model_name = "google-t5/t5-small"  # 60M parámetros
            batch_size = 8
            print("   → Usando T5-small (60M parámetros) con batch size 8")
    else:
        print("⚠️  No hay GPU disponible - Usando CPU (muy lento)")
        model_name = "google-t5/t5-small"
        batch_size = 2

    # Información de RAM
    import psutil
    ram_gb = psutil.virtual_memory().total / 1e9
    ram_available = psutil.virtual_memory().available / 1e9
    print(f"\n💿 RAM Total: {ram_gb:.2f} GB")
    print(f"💿 RAM Disponible: {ram_available:.2f} GB")

    # Espacio en disco
    import shutil
    total, used, free = shutil.disk_usage("/")
    print(f"📁 Disco total: {total / 1e9:.1f} GB")
    print(f"📁 Disco libre: {free / 1e9:.1f} GB")

    return model_name, batch_size, gpu_info

MODEL_NAME, BASE_BATCH_SIZE, GPU_INFO = check_resources_and_setup()


🔍 VERIFICACIÓN DE RECURSOS - Google Colab Pro
🎯 GPU: NVIDIA L4
💾 Memoria GPU total: 23.80 GB
💾 Memoria GPU disponible: 23.80 GB
   → Usando T5-base (220M parámetros) con batch size 4

💿 RAM Total: 56.86 GB
💿 RAM Disponible: 54.45 GB
📁 Disco total: 253.1 GB
📁 Disco libre: 210.4 GB


Se cofiguran los parámetros seguros y demás. Es aquí cuando me di cuenta de lo conveniente que es tener librerías como Unsloth...

In [ ]:
# ============================================================================
# CELDA 3: CONFIGURACIÓN DE PARÁMETROS SEGUROS
# ============================================================================
class SafeT5TrainingConfig:
    """Configuración segura para fine-tuning de T5"""

    # ========== MODELO ==========
    model_name = MODEL_NAME
    tokenizer_name = MODEL_NAME

    # ========== DATASET ==========
    # Cambia estas rutas según tu estructura
    dataset_base_path = "/content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_t5_final"
    train_path = f"{dataset_base_path}/train.jsonl"
    val_path = f"{dataset_base_path}/val.jsonl"
    test_path = f"{dataset_base_path}/test.jsonl"

    # ========== HIPERPARÁMETROS SEGUROS ==========
    max_input_length = 256  # Reducido para seguridad
    max_target_length = 128
    batch_size = BASE_BATCH_SIZE
    gradient_accumulation_steps = max(1, 8 // BASE_BATCH_SIZE)  # Ajuste automático
    num_train_epochs = 3
    learning_rate = 2e-4  # Un poco más bajo para estabilidad
    weight_decay = 0.01
    warmup_steps = 50

    # ========== ENTRENAMIENTO SEGURO ==========
    logging_steps = 25
    eval_steps = 100  # Evaluar cada 100 pasos
    save_steps = 200
    save_total_limit = 2
    load_best_model_at_end = True
    metric_for_best_model = "eval_loss"
    greater_is_better = False

    # ========== GENERACIÓN (DESACTIVADA DURANTE EVAL) ==========
    num_beams = 4
    temperature = 0.7
    top_p = 0.9
    repetition_penalty = 1.2

    # ========== SALIDA ==========
    output_dir = f"/content/drive/MyDrive/Practica_LLM_Engineering_25/t5_electoral_safe_{datetime.now().strftime('%Y%m%d_%H%M')}"
    report_to = "none"

    # ========== SEGURIDAD ==========
    use_safe_tokenization = True  # Usar tokenización segura
    filter_problematic_examples = True  # Filtrar ejemplos problemáticos
    predict_with_generate_during_training = False  # ¡IMPORTANTE! Desactivar generación durante eval

    def __str__(self):
        config_str = "\n⚙️ CONFIGURACIÓN DE ENTRENAMIENTO SEGURO:\n"
        config_str += "=" * 50 + "\n"

        for key, value in self.__dict__.items():
            if not key.startswith('_'):
                config_str += f"{key:35}: {value}\n"

        return config_str

config = SafeT5TrainingConfig()
print(config)



⚙️ CONFIGURACIÓN DE ENTRENAMIENTO SEGURO:



Cargamos los tres datasests de train, test y validación, y se muestran una serie de ejemplos.

In [ ]:
# ============================================================================
# CELDA 4: CARGA Y VERIFICACIÓN DEL DATASET
# ============================================================================
from datasets import Dataset, DatasetDict
import hashlib

print("\n" + "=" * 70)
print("📂 CARGA Y VERIFICACIÓN DEL DATASET")
print("=" * 70)

def load_and_validate_dataset(config):
    """Carga y valida el dataset cuidadosamente"""

    def read_jsonl_safely(filepath, max_examples=None):
        """Lee JSONL con validación robusta"""
        data = []
        errors = []

        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                for line_num, line in enumerate(f, 1):
                    if max_examples and len(data) >= max_examples:
                        break

                    line = line.strip()
                    if not line:
                        continue

                    try:
                        example = json.loads(line)

                        # Validar estructura
                        if not isinstance(example, dict):
                            errors.append(f"Línea {line_num}: No es un diccionario")
                            continue

                        # Verificar campos requeridos
                        if 'input_text' not in example or 'target_text' not in example:
                            errors.append(f"Línea {line_num}: Faltan campos requeridos")
                            continue

                        # Verificar que no estén vacíos
                        if not example['input_text'] or not example['target_text']:
                            errors.append(f"Línea {line_num}: Campos vacíos")
                            continue

                        # Limpiar textos
                        input_text = str(example['input_text']).strip()
                        target_text = str(example['target_text']).strip()

                        # Añadir ID único
                        _id = hashlib.md5(
                            f"{input_text}_{target_text}".encode()
                        ).hexdigest()[:8]

                        # Only append the cleaned and necessary fields to ensure consistent schema
                        data.append({
                            'input_text': input_text,
                            'target_text': target_text,
                            '_id': _id
                        })

                    except json.JSONDecodeError as e:
                        errors.append(f"Línea {line_num}: JSON inválido - {str(e)}")
                    except Exception as e:
                        errors.append(f"Línea {line_num}: Error - {str(e)}")

            print(f"  ✅ {len(data)} ejemplos válidos")
            if errors:
                print(f"  ⚠️  {len(errors)} errores (primeros 3):")
                for err in errors[:3]:
                    print(f"     {err}")

            return data

        except FileNotFoundError:
            print(f"  ❌ Archivo no encontrado: {filepath}")
            return []
        except Exception as e:
            print(f"  ❌ Error cargando {filepath}: {e}")
            return []

    print("Cargando splits...")

    # Cargar con límite si es muy grande
    train_data = read_jsonl_safely(config.train_path)
    val_data = read_jsonl_safely(config.val_path) if os.path.exists(config.val_path) else []
    test_data = read_jsonl_safely(config.test_path) if os.path.exists(config.test_path) else []

    if not train_data:
        raise ValueError("❌ No se pudo cargar el dataset de entrenamiento")

    # Crear DatasetDict
    dataset_dict = {}
    dataset_dict["train"] = Dataset.from_list(train_data)

    if val_data:
        dataset_dict["validation"] = Dataset.from_list(val_data)

    if test_data:
        dataset_dict["test"] = Dataset.from_list(test_data)

    dataset_dict = DatasetDict(dataset_dict)

    # Estadísticas
    print(f"\n📊 ESTADÍSTICAS DEL DATASET:")
    print(f"  Entrenamiento: {len(train_data)} ejemplos")
    if val_data:
        print(f"  Validación: {len(val_data)} ejemplos")
    if test_data:
        print(f"  Test: {len(test_data)} ejemplos")

    # Mostrar ejemplos de muestra
    print(f"\n📝 MUESTRA DEL DATASET (3 ejemplos):")
    for i in range(min(3, len(train_data))):
        example = train_data[i]
        print(f"\n  Ejemplo {i+1} (ID: {example.get('_id', 'N/A')}):")
        print(f"  Input: {example['input_text'][:80]}...")
        print(f"  Target: {example['target_text'][:80]}...")
        if 'metadata' in example:
            nivel = example['metadata'].get('nivel_original', 'N/A')
            dificultad = example['metadata'].get('dificultad', 'N/A')
            print(f"  Nivel: {nivel}, Dificultad: {dificultad}")

    return dataset_dict

# Cargar dataset
dataset = load_and_validate_dataset(config)



📂 CARGA Y VERIFICACIÓN DEL DATASET
Cargando splits...
  ✅ 1440 ejemplos válidos
  ✅ 180 ejemplos válidos
  ✅ 180 ejemplos válidos

📊 ESTADÍSTICAS DEL DATASET:
  Entrenamiento: 1440 ejemplos
  Validación: 180 ejemplos
  Test: 180 ejemplos

📝 MUESTRA DEL DATASET (3 ejemplos):

  Ejemplo 1 (ID: 1863b614):
  Input: ### Instrucción:
¿Cuál fue el % VOX mediano en la comunidad de Andalucía en Novi...
  Target: El % VOX mediano en la comunidad de Andalucía en Noviembre 2019 fue de 19.90%....

  Ejemplo 2 (ID: 86113f39):
  Input: ### Instrucción:
¿Qué diferencia hay entre PP y VOX en la provincia de Segovia e...
  Target: En la elección de Abril 2019 en la provincia de Segovia, el partido PP obtuvo el...

  Ejemplo 3 (ID: 2fcd4bb0):
  Input: ### Instrucción:
¿Cómo evolucionó el porcentaje de voto a VOX en Cillorigo de Li...
  Target: El VOX en Cillorigo de Liébana pasó de 19.61% en Noviembre 2019 a 15.03% en Abri...


Llegamos a la tokenizacion que, pese a cargar el tokenizador desde transformers, tuve continuos problemas con él. Tras las rectificaciones de DeepSeek, Colab estuvo cerca de una hora intentando solucionar el problema.

In [ ]:
# ============================================================================
# CELDA 5: TOKENIZACIÓN SEGURA PARA T5
# ============================================================================
from transformers import T5Tokenizer
import re

print("\n" + "=" * 70)
print("🔤 TOKENIZACIÓN SEGURA DEL DATASET")
print("=" * 70)

print(f"Cargando tokenizer: {config.tokenizer_name}")
tokenizer = T5Tokenizer.from_pretrained(config.tokenizer_name)

# Configurar tokenizer para T5
tokenizer.pad_token = tokenizer.eos_token

print(f"  Vocabulario: {tokenizer.vocab_size:,} tokens")
print(f"  Pad token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"  EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")

# ============================================================================
# FUNCIÓN DE PREPROCESAMIENTO SEGURA
# ============================================================================
def safe_preprocess_function(examples, is_validation=False):
    """
    Función de preprocesamiento SEGURA con verificación exhaustiva
    """
    # Limpiar textos
    def clean_text(text):
        if not isinstance(text, str):
            return ""

        # 1. Normalizar espacios
        text = ' '.join(text.split())

        # 2. Mantener solo caracteres seguros
        # Permitir: letras, números, espacios, puntuación básica, acentos españoles
        safe_pattern = r'[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑüÜ\s.,;:¿?¡!()\-%\d]'
        text = re.sub(safe_pattern, '', text)

        # 3. Limitar longitud
        max_chars = config.max_input_length * 4  # Estimación conservadora
        if len(text) > max_chars:
            text = text[:max_chars]

        return text.strip()

    # Limpiar inputs y targets
    inputs = [clean_text(text) for text in examples["input_text"]]
    targets = [clean_text(text) for text in examples["target_text"]]

    # Tokenizar inputs
    model_inputs = tokenizer(
        inputs,
        max_length=config.max_input_length,
        truncation=True,
        padding="max_length",
        return_tensors=None
    )

    # Tokenizar targets con contexto de target tokenizer
    with tokenizer.as_target_tokenizer():
        tokenized_targets = tokenizer(
            targets,
            max_length=config.max_target_length,
            truncation=True,
            padding="max_length",
            return_tensors=None
        )

    # VERIFICACIÓN Y CORRECCIÓN EXHAUSTIVA DE LABELS
    labels = tokenized_targets["input_ids"]

    for i in range(len(labels)):
        for j in range(len(labels[i])):
            token_id = labels[i][j]

            # 1. Verificar que sea un número entero
            if not isinstance(token_id, int):
                labels[i][j] = tokenizer.pad_token_id
                continue

            # 2. Verificar rango del vocabulario (0 a vocab_size-1)
            if token_id < 0 or token_id >= tokenizer.vocab_size:
                labels[i][j] = tokenizer.pad_token_id
                continue

            # 3. Reemplazar padding token por -100 (ignore index)
            if token_id == tokenizer.pad_token_id:
                labels[i][j] = -100

    model_inputs["labels"] = labels

    # Verificación de seguridad (solo para primeros ejemplos)
    if not is_validation and len(model_inputs["input_ids"]) > 0:
        # Verificar primer ejemplo
        first_input_ids = model_inputs["input_ids"][0]
        first_labels = model_inputs["labels"][0]

        max_input_id = max(first_input_ids)
        valid_labels = [l for l in first_labels if l != -100]
        max_label_id = max(valid_labels) if valid_labels else 0

        if max_input_id >= tokenizer.vocab_size:
            print(f"⚠️  ADVERTENCIA: input_ids tiene ID {max_input_id} >= {tokenizer.vocab_size}")

        if valid_labels and max_label_id >= tokenizer.vocab_size:
            print(f"⚠️  ADVERTENCIA: labels tiene ID {max_label_id} >= {tokenizer.vocab_size}")

    return model_inputs

# ============================================================================
# DIAGNÓSTICO DE PROBLEMAS EN VALIDATION SET
# ============================================================================
print("\n" + "=" * 70)
print("🔍 DIAGNÓSTICO PREVENTIVO DE VALIDATION SET")
print("=" * 70)

def diagnose_validation_set(validation_dataset):
    """Diagnostica y corrige problemas en el validation set"""

    if not validation_dataset:
        print("⚠️  No hay validation set para diagnosticar")
        return None

    print(f"Analizando validation set ({len(validation_dataset)} ejemplos)...")

    # Tokenizar validation set para diagnóstico
    val_tokenized_for_diagnosis = validation_dataset.map(
        lambda x: safe_preprocess_function(x, is_validation=True),
        batched=True,
        batch_size=32,
        remove_columns=validation_dataset.column_names
    )

    # Buscar ejemplos problemáticos
    problematic_indices = []
    problematic_details = []

    for i in range(len(val_tokenized_for_diagnosis)):
        example = val_tokenized_for_diagnosis[i]

        # Verificar input_ids
        input_ids = example["input_ids"]
        if any(token_id >= tokenizer.vocab_size for token_id in input_ids):
            problematic_indices.append(i)
            problematic_details.append({
                "index": i,
                "issue": "input_ids out of range",
                "max_id": max(input_ids),
                "vocab_size": tokenizer.vocab_size
            })
            continue

        # Verificar labels
        labels = example["labels"]
        valid_labels = [l for l in labels if l != -100]

        if valid_labels:
            if any(label_id >= tokenizer.vocab_size for label_id in valid_labels):
                problematic_indices.append(i)
                problematic_details.append({
                    "index": i,
                    "issue": "labels out of range",
                    "max_id": max(valid_labels),
                    "vocab_size": tokenizer.vocab_size
                })

    if problematic_indices:
        print(f"❌ ENCONTRADOS {len(problematic_indices)} EJEMPLOS PROBLEMÁTICOS")
        print("Detalles (primeros 5):")
        for detail in problematic_details[:5]:
            print(f"  Índice {detail['index']}: {detail['issue']}")
            print(f"     ID máximo: {detail['max_id']}, Vocab size: {detail['vocab_size']}")

        # Mostrar textos problemáticos
        print(f"\n📝 Textos de ejemplos problemáticos:")
        for idx in problematic_indices[:3]:
            original_example = validation_dataset[idx]
            print(f"\n  Índice {idx}:")
            print(f"  Input: {original_example['input_text'][:100]}...")
            print(f"  Target: {original_example['target_text'][:100]}...")

        if config.filter_problematic_examples:
            print(f"\n🛠️  FILTRANDO EJEMPLOS PROBLEMÁTICOS...")

            # Crear nuevo validation set sin ejemplos problemáticos
            good_indices = [i for i in range(len(validation_dataset)) if i not in problematic_indices]

            from datasets import Dataset
            good_examples = [validation_dataset[i] for i in good_indices]

            print(f"  Original: {len(validation_dataset)} ejemplos")
            print(f"  Filtrado: {len(good_examples)} ejemplos")
            print(f"  Eliminados: {len(problematic_indices)} ejemplos")

            return Dataset.from_list(good_examples)
        else:
            print("⚠️  Manteniendo ejemplos problemáticos (puede causar errores)")
            return validation_dataset
    else:
        print("✅ Validation set verificado - No se encontraron problemas")
        return validation_dataset

# Aplicar diagnóstico al validation set si existe
if "validation" in dataset:
    print("\nAplicando diagnóstico al validation set...")
    dataset["validation"] = diagnose_validation_set(dataset["validation"])

# ============================================================================
# TOKENIZACIÓN FINAL
# ============================================================================
print("\n🔄 TOKENIZACIÓN FINAL DE DATASETS...")

def safe_tokenize_dataset(dataset_split, split_name):
    """Tokeniza un split del dataset de forma segura"""
    print(f"  Tokenizando {split_name}...")

    is_validation = (split_name == "validation")

    tokenized = dataset_split.map(
        lambda x: safe_preprocess_function(x, is_validation=is_validation),
        batched=True,
        batch_size=32,
        remove_columns=dataset_split.column_names
    )

    print(f"    ✅ {len(tokenized)} ejemplos tokenizados")
    return tokenized

tokenized_datasets = {}

# Tokenizar train
tokenized_datasets["train"] = safe_tokenize_dataset(dataset["train"], "train")

# Tokenizar validation si existe y no está vacío
if "validation" in dataset and len(dataset["validation"]) > 0:
    tokenized_datasets["validation"] = safe_tokenize_dataset(dataset["validation"], "validation")
else:
    print("⚠️  Validation set no disponible o vacío")
    config.eval_steps = None  # Desactivar evaluación

# Tokenizar test si existe
if "test" in dataset and len(dataset["test"]) > 0:
    tokenized_datasets["test"] = safe_tokenize_dataset(dataset["test"], "test")

print(f"\n✅ TOKENIZACIÓN COMPLETADA:")
print(f"  Train: {len(tokenized_datasets['train'])} ejemplos")
if "validation" in tokenized_datasets:
    print(f"  Validation: {len(tokenized_datasets['validation'])} ejemplos")
if "test" in tokenized_datasets:
    print(f"  Test: {len(tokenized_datasets['test'])} ejemplos")


🔤 TOKENIZACIÓN SEGURA DEL DATASET
Cargando tokenizer: google-t5/t5-base


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  Vocabulario: 32,000 tokens
  Pad token: </s> (ID: 1)
  EOS token: </s> (ID: 1)

🔍 DIAGNÓSTICO PREVENTIVO DE VALIDATION SET

Aplicando diagnóstico al validation set...
Analizando validation set (180 ejemplos)...


Map:   0%|          | 0/180 [00:00<?, ? examples/s]

✅ Validation set verificado - No se encontraron problemas

🔄 TOKENIZACIÓN FINAL DE DATASETS...
  Tokenizando train...


...

    ✅ 1440 ejemplos tokenizados
  Tokenizando validation...


Map:   0%|          | 0/180 [00:00<?, ? examples/s]

    ✅ 180 ejemplos tokenizados
  Tokenizando test...


Map:   0%|          | 0/180 [00:00<?, ? examples/s]

    ✅ 180 ejemplos tokenizados

✅ TOKENIZACIÓN COMPLETADA:
  Train: 1440 ejemplos
  Validation: 180 ejemplos
  Test: 180 ejemplos


Cargamos el modelo T5 desde transformers, esta celda fue quizás la única que no dio problemas.

In [ ]:
# ============================================================================
# CELDA 6: CARGA DEL MODELO T5
# ============================================================================
from transformers import T5ForConditionalGeneration

print("\n" + "=" * 70)
print("🏗️  CARGA DEL MODELO T5")
print("=" * 70)

print(f"Cargando modelo: {config.model_name}")

try:
    # Configuración del modelo
    model = T5ForConditionalGeneration.from_pretrained(config.model_name)

    # Mover a GPU si está disponible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    print(f"✅ Modelo cargado en: {device}")
    print(f"   Parámetros totales: {sum(p.numel() for p in model.parameters()):,}")
    print(f"   Parámetros entrenables: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

except Exception as e:
    print(f"❌ Error cargando el modelo: {e}")
    print("Intentando cargar versión alternativa...")

    # Intentar con configuraciones alternativas
    try:
        model = T5ForConditionalGeneration.from_pretrained(
            config.model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )
        model = model.to(device)
        print("✅ Modelo cargado con configuración alternativa")
    except Exception as e2:
        print(f"❌ Error crítico: {e2}")
        raise



🏗️  CARGA DEL MODELO T5
Cargando modelo: google-t5/t5-base
✅ Modelo cargado en: cuda
   Parámetros totales: 222,903,552
   Parámetros entrenables: 222,903,552


Esta celda configura las métricas, de forma 'segura'. Esto de 'seguro' es una constante de DeepSeek, tras informarle yo de los problemas que iba encontrando.

De todas formas, soy algo escéptico, quizás debido a mi ignorancia, con esta mediciones de Rouge, al menos en esta práctica. Lo explico en el pdf de presentación de la práctica.

In [ ]:
# ============================================================================
# CELDA 7: CONFIGURACIÓN DE MÉTRICAS SEGURAS
# ============================================================================
import evaluate
import nltk

print("\n" + "=" * 70)
print("📊 CONFIGURACIÓN DE MÉTRICAS SEGURAS")
print("=" * 70)

try:
    nltk.download('punkt', quiet=True)

    # Cargar métricas
    rouge = evaluate.load("rouge")

    def safe_compute_metrics(eval_pred):
        """
        Calcula métricas de forma segura
        Solo se usará si predict_with_generate=True
        """
        predictions, labels = eval_pred

        # Verificar que las predicciones sean válidas
        if predictions is None or labels is None:
            return {"eval_loss": 0.0}

        # Decodificar predicciones
        try:
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        except:
            decoded_preds = [""] * len(predictions)

        # Reemplazar -100 en labels
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        try:
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        except:
            decoded_labels = [""] * len(labels)

        # Calcular ROUGE de forma segura
        try:
            rouge_result = rouge.compute(
                predictions=decoded_preds,
                references=decoded_labels,
                use_stemmer=True,
                use_aggregator=True
            )
        except:
            rouge_result = {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0, "rougeLsum": 0.0}

        return {
            "rouge1": rouge_result["rouge1"],
            "rouge2": rouge_result["rouge2"],
            "rougeL": rouge_result["rougeL"],
            "rougeLsum": rouge_result["rougeLsum"]
        }

    print("✅ Métricas configuradas (solo para evaluación final)")

except Exception as e:
    print(f"⚠️  Error configurando métricas: {e}")
    print("Continuando sin métricas detalladas")

    def safe_compute_metrics(eval_pred):
        return {"eval_loss": 0.0}


📊 CONFIGURACIÓN DE MÉTRICAS SEGURAS
✅ Métricas configuradas (solo para evaluación final)


Ahora configuramos el finetuning, de nuevo de forma 'segura', con la librería de transformers. La mayor parte de los parámetros los hemos definido en la celda 3, con el objeto creado allí de la clase SafeT5TrainingConfig, llamado 'config'.

Por fin, definimos el trainer con la función Seq2SeqTrainer().

In [ ]:
# ============================================================================
# CELDA 8: CONFIGURACIÓN DEL ENTRENAMIENTO SEGURO
# ============================================================================
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

print("\n" + "=" * 70)
print("🎯 CONFIGURACIÓN DEL ENTRENAMIENTO SEGURO")
print("=" * 70)

# Data collator para T5
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

# Configurar si hay validation set
has_validation = "validation" in tokenized_datasets and len(tokenized_datasets["validation"]) > 0

# Argumentos de entrenamiento SEGUROS
training_args = Seq2SeqTrainingArguments(
    output_dir=config.output_dir,
    overwrite_output_dir=True,

    # Hiperparámetros
    num_train_epochs=config.num_train_epochs,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    learning_rate=config.learning_rate,
    weight_decay=config.weight_decay,
    warmup_steps=config.warmup_steps,

    # Estrategias SEGURAS
    evaluation_strategy="steps" if has_validation else "no",
    eval_steps=config.eval_steps if has_validation else None,
    save_strategy="steps",
    save_steps=config.save_steps,
    save_total_limit=config.save_total_limit,
    load_best_model_at_end=has_validation,
    metric_for_best_model=config.metric_for_best_model if has_validation else None,
    greater_is_better=config.greater_is_better if has_validation else None,

    # Logging y reporte
    logging_strategy="steps",
    logging_steps=config.logging_steps,
    report_to=config.report_to,

    # Optimización SEGURA
    fp16=torch.cuda.is_available(),
    optim="adafactor",  # Adafactor es más estable para T5
    gradient_checkpointing=True,

    # ⚠️ CONFIGURACIÓN CRÍTICA: Generación durante evaluación
    # Desactivar para evitar error "piece id is out of range"
    predict_with_generate=config.predict_with_generate_during_training and has_validation,

    # Configuración de generación (solo si predict_with_generate=True)
    generation_max_length=config.max_target_length if config.predict_with_generate_during_training else None,
    generation_num_beams=config.num_beams if config.predict_with_generate_during_training else None,

    # Otros ajustes de seguridad
    dataloader_num_workers=0,  # 0 para evitar problemas en Colab
    remove_unused_columns=True,
    push_to_hub=False,
    ddp_find_unused_parameters=False,
)

print("✅ Training arguments configurados SEGUROS")
print(f"   Output directory: {config.output_dir}")
print(f"   Batch size: {config.batch_size}")
print(f"   Gradient accumulation: {config.gradient_accumulation_steps}")
print(f"   Total effective batch: {config.batch_size * config.gradient_accumulation_steps}")
print(f"   Evaluation: {'Activada' if has_validation else 'Desactivada'}")
print(f"   Predict with generate: {training_args.predict_with_generate}")

# Crear trainer SEGURO
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets.get("validation", None),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=safe_compute_metrics if training_args.predict_with_generate else None,
)

print("✅ Trainer creado exitosamente")


🎯 CONFIGURACIÓN DEL ENTRENAMIENTO SEGURO
✅ Training arguments configurados SEGUROS
   Output directory: /content/drive/MyDrive/Practica_LLM_Engineering_25/t5_electoral_safe_20260203_1558
   Batch size: 4
   Gradient accumulation: 2
   Total effective batch: 8
   Evaluation: Activada
   Predict with generate: False
✅ Trainer creado exitosamente


Ejecutamos el finetuning, que duró unos 5-7 minutos. Lo cierto es que notamos que los errores van descendiendo a lo largo de las tres época, tanto en train como validación.

In [ ]:
# ============================================================================
# CELDA 9: ENTRENAMIENTO SEGURO CON MANEJO DE ERRORES
# ============================================================================
print("\n" + "=" * 70)
print("🚀 INICIANDO ENTRENAMIENTO SEGURO")
print("=" * 70)

# Limpiar memoria antes de empezar
torch.cuda.empty_cache()
gc.collect()

# Mostrar memoria disponible
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1e9
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"Memoria GPU antes del entrenamiento: {allocated:.2f} GB / {total:.2f} GB")

# Calcular pasos totales estimados
total_steps = len(tokenized_datasets["train"]) // (config.batch_size * config.gradient_accumulation_steps) * config.num_train_epochs
print(f"Pasos totales estimados: {total_steps}")
print(f"Duración estimada: {total_steps * 2 / 60:.1f} minutos (aproximadamente)")

# Función de callback para manejar errores
from transformers import TrainerCallback

class SafeTrainingCallback(TrainerCallback):
    """Callback para manejar errores durante el entrenamiento"""

    def on_step_end(self, args, state, control, **kwargs):
        # Verificar cada 50 pasos
        if state.global_step % 50 == 0:
            torch.cuda.empty_cache()

        return control

    def on_evaluate(self, args, state, control, **kwargs):
        print(f"\n🔍 Evaluación completada en paso {state.global_step}")
        return control

# Añadir callback al trainer
trainer.add_callback(SafeTrainingCallback())

# INTENTAR ENTRENAR CON MANEJO DE ERRORES
try:
    print("\n" + "=" * 70)
    print("⏳ ENTRENANDO... (esto puede tomar varios minutos)")
    print("=" * 70)

    # Entrenar
    train_result = trainer.train()

    # Guardar modelo final
    trainer.save_model()
    tokenizer.save_pretrained(config.output_dir)

    # Guardar métricas de entrenamiento
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

    print(f"\n" + "=" * 70)
    print("🎉 ¡ENTRENAMIENTO COMPLETADO EXITOSAMENTE!")
    print("=" * 70)
    print(f"📁 Modelo guardado en: {config.output_dir}")

    # Mostrar resumen
    print(f"\n📊 RESUMEN DEL ENTRENAMIENTO:")
    print(f"  Épocas completadas: {config.num_train_epochs}")
    print(f"  Pasos totales: {train_result.global_step}")
    print(f"  Pérdida final: {train_result.training_loss:.4f}")

    if has_validation:
        print(f"  Pérdida en validación: {trainer.state.log_history[-1].get('eval_loss', 'N/A')}")

except RuntimeError as e:
    if "out of memory" in str(e):
        print("\n❌ ERROR: Memoria insuficiente (OOM)")
        print("Intentando recuperación...")

        # Guardar lo que se haya logrado
        try:
            trainer.save_model(config.output_dir + "_partial")
            print(f"✅ Modelo parcial guardado en: {config.output_dir}_partial")
        except:
            print("⚠️  No se pudo guardar modelo parcial")

        print("\n💡 SOLUCIONES:")
        print("1. Reduce batch_size a la mitad")
        print("2. Reduce max_input_length y max_target_length")
        print("3. Usa gradient_checkpointing=True")
        print("4. Usa T5-small en lugar de T5-base")

    elif "piece id is out of range" in str(e):
        print("\n❌ ERROR: Token ID fuera de rango durante evaluación")
        print("Esto probablemente ocurrió durante la evaluación con generación.")

        print("\n🛠️  SOLUCIÓN APLICADA AUTOMÁTICAMENTE:")
        print("Reentrenando SIN generación durante evaluación...")

        # Reconfigurar para entrenar sin generación
        training_args.predict_with_generate = False

        # Crear nuevo trainer
        trainer_no_generate = Seq2SeqTrainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets.get("validation", None),
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=None,  # Sin métricas que requieran generación
        )

        # Entrenar de nuevo
        print("Reiniciando entrenamiento sin generación durante evaluación...")
        train_result = trainer_no_generate.train()

        # Guardar
        trainer_no_generate.save_model()
        tokenizer.save_pretrained(config.output_dir + "_no_generate")

        print(f"\n✅ Entrenamiento completado sin generación")
        print(f"📁 Modelo guardado en: {config.output_dir}_no_generate")

    else:
        print(f"\n❌ ERROR durante el entrenamiento: {e}")
        import traceback
        traceback.print_exc()

except Exception as e:
    print(f"\n❌ ERROR inesperado: {e}")
    import traceback
    traceback.print_exc()


🚀 INICIANDO ENTRENAMIENTO SEGURO
Memoria GPU antes del entrenamiento: 0.89 GB / 23.80 GB
Pasos totales estimados: 540
Duración estimada: 18.0 minutos (aproximadamente)

⏳ ENTRENANDO... (esto puede tomar varios minutos)


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
100,1.038000,0.807598
200,0.690500,0.674657
300,0.707100,0.616378
400,0.565600,0.593632
500,0.522900,0.583102



🔍 Evaluación completada en paso 100

🔍 Evaluación completada en paso 200

🔍 Evaluación completada en paso 300

🔍 Evaluación completada en paso 400

🔍 Evaluación completada en paso 500


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


***** train metrics *****
  epoch                    =        3.0
  total_flos               =  1225014GF
  train_loss               =     0.8151
  train_runtime            = 0:04:30.71
  train_samples_per_second =     15.958
  train_steps_per_second   =      1.995

🎉 ¡ENTRENAMIENTO COMPLETADO EXITOSAMENTE!
📁 Modelo guardado en: /content/drive/MyDrive/Practica_LLM_Engineering_25/t5_electoral_safe_20260203_1558

📊 RESUMEN DEL ENTRENAMIENTO:
  Épocas completadas: 3
  Pasos totales: 540
  Pérdida final: 0.8151
  Pérdida en validación: N/A


Probamos el modelo T5 ya finetuneado con el dataset de Test. Obtenemos unas pérdidas bastante parecidas a las de train y validation, 0,57, con lo que no parece haber un overfitting claro.

Vemos que obtenemos un resultado de Rouge de cero, lo cual pregunté al respecto a DeepSeek.

In [ ]:
# ============================================================================
# CELDA 10: EVALUACIÓN FINAL SEGURA
# ============================================================================
print("\n" + "=" * 70)
print("📊 EVALUACIÓN FINAL SEGURA")
print("=" * 70)

if "test" in tokenized_datasets and len(tokenized_datasets["test"]) > 0:
    print("Evaluando en conjunto de test...")

    try:
        # Para evaluación final, podemos usar generación
        # pero con configuración segura

        # Configurar generación segura para evaluación final
        eval_args = Seq2SeqTrainingArguments(
            output_dir=config.output_dir + "_final_eval",
            per_device_eval_batch_size=config.batch_size,
            predict_with_generate=True,  # Usar generación solo para evaluación final
            generation_max_length=config.max_target_length,
            generation_num_beams=1,  # Usar beam=1 para mayor estabilidad
            dataloader_num_workers=0,
        )

        eval_trainer = Seq2SeqTrainer(
            model=model,
            args=eval_args,
            eval_dataset=tokenized_datasets["test"],
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=safe_compute_metrics,
        )

        print("Realizando evaluación final con generación...")
        eval_results = eval_trainer.evaluate()

        print("\n📈 RESULTADOS EN TEST:")
        print("-" * 40)
        for key, value in eval_results.items():
            if isinstance(value, float):
                print(f"{key:20}: {value:.4f}")

        # Guardar resultados
        with open(f"{config.output_dir}/test_results.json", "w") as f:
            json.dump(eval_results, f, indent=2)

        print(f"✅ Resultados guardados en: {config.output_dir}/test_results.json")

    except Exception as e:
        print(f"⚠️  Error en evaluación final: {e}")
        print("Realizando evaluación simple (solo loss)...")

        # Evaluación simple sin generación
        try:
            simple_eval_args = Seq2SeqTrainingArguments(
                output_dir=config.output_dir + "_simple_eval",
                per_device_eval_batch_size=config.batch_size,
                predict_with_generate=False,
                dataloader_num_workers=0,
            )

            simple_eval_trainer = Seq2SeqTrainer(
                model=model,
                args=simple_eval_args,
                eval_dataset=tokenized_datasets["test"],
                tokenizer=tokenizer,
                data_collator=data_collator,
            )

            simple_results = simple_eval_trainer.evaluate()
            print(f"📊 Pérdida en test: {simple_results['eval_loss']:.4f}")

        except Exception as e2:
            print(f"⚠️  Error incluso en evaluación simple: {e2}")
else:
    print("⚠️  No hay conjunto de test para evaluación")



📊 EVALUACIÓN FINAL SEGURA
Evaluando en conjunto de test...
Realizando evaluación final con generación...


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin



📈 RESULTADOS EN TEST:
----------------------------------------
eval_loss           : 0.5740
eval_rouge1         : 0.0000
eval_rouge2         : 0.0000
eval_rougeL         : 0.0000
eval_rougeLsum      : 0.0000
eval_runtime        : 142.2180
eval_samples_per_second: 1.2660
eval_steps_per_second: 0.3160
✅ Resultados guardados en: /content/drive/MyDrive/Practica_LLM_Engineering_25/t5_electoral_safe_20260203_1558/test_results.json


DeepSeek añadió esta celda para probar el modelo con preguntas/respuestas que no hubiese visto el modelo antes. Honestamente, no pensé que fuese interesante cuando no hemos siquiera configurar el RAG, con lo que no la ejecuté.

In [ ]:
# ============================================================================
# CELDA 11: PRUEBA DEL MODELO ENTRENADO
# ============================================================================
print("\n" + "=" * 70)
print("🧪 PRUEBA DEL MODELO ENTRENADO")
print("=" * 70)

def test_trained_model(model_path, test_questions, max_examples=3):
    """Prueba el modelo con preguntas de ejemplo"""

    from transformers import T5Tokenizer, T5ForConditionalGeneration
    import torch

    print(f"Cargando modelo desde {model_path}...")

    try:
        tokenizer_test = T5Tokenizer.from_pretrained(model_path)
        model_test = T5ForConditionalGeneration.from_pretrained(model_path)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model_test = model_test.to(device)
        model_test.eval()

        print(f"✅ Modelo cargado en {device}")

    except Exception as e:
        print(f"❌ Error cargando modelo: {e}")
        return

    print("\n📝 PROBANDO CON PREGUNTAS DE EJEMPLO:")

    for i, question in enumerate(test_questions[:max_examples]):
        print(f"\n--- Ejemplo {i+1} ---")
        print(f"Pregunta: {question}")

        # Asegurar formato correcto
        if not question.strip().endswith("respuesta:"):
            question = question.strip() + " respuesta:"

        try:
            # Tokenizar
            inputs = tokenizer_test(
                question,
                return_tensors="pt",
                truncation=True,
                max_length=config.max_input_length,
                padding=True
            ).to(device)

            # Generar
            with torch.no_grad():
                outputs = model_test.generate(
                    **inputs,
                    max_length=config.max_target_length,
                    num_beams=config.num_beams,
                    temperature=config.temperature,
                    do_sample=True,
                    repetition_penalty=config.repetition_penalty,
                    early_stopping=True
                )

            # Decodificar
            response = tokenizer_test.decode(outputs[0], skip_special_tokens=True)

            # Extraer solo la respuesta (lo que viene después del prompt)
            prompt_length = len(tokenizer_test.decode(inputs.input_ids[0], skip_special_tokens=True))
            response_only = response[prompt_length:].strip()

            print(f"🤖 Respuesta: {response_only}")

        except Exception as e:
            print(f"❌ Error generando respuesta: {e}")

# Preguntas de prueba
test_questions = [
    "pregunta: ¿Qué porcentaje de votos obtuvo el PSOE en el municipio de Madrid en 2019?",
    "pregunta: ¿Cómo evolucionó la participación electoral entre 2016 y 2019?",
    "pregunta: ¿Cuál fue el partido más votado en las últimas elecciones?",
]

# Probar el modelo
test_trained_model(config.output_dir, test_questions)


Después de preguntarle a DeepSeek por el cero obtenido con Rouge, me recomendó esta celda adicional, donde examina ejemplos de test, y que obtiene coeficientes de Rouge superiores a cero. El problema es que en esos casos recoge en la respuesta datos que le hemos pasado en la pregunta, con lo que, sinceramente, no me parece útil esta métrica.

In [ ]:
# AÑADE ESTA CELDA PARA DIAGNOSTICAR ROUGE
print("\n" + "=" * 70)
print("🔍 DIAGNÓSTICO DE MÉTRICAS ROUGE")
print("=" * 70)

def debug_rouge_metrics(model_path, test_dataset, num_examples=5):
    """Debug detallado de las métricas ROUGE"""

    from transformers import T5Tokenizer, T5ForConditionalGeneration
    import torch

    # Cargar modelo
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    print(f"Analizando {num_examples} ejemplos del test set...\n")

    for i in range(min(num_examples, len(test_dataset))):
        example = test_dataset[i]

        print(f"\n--- Ejemplo {i+1} ---")
        print(f"📝 INPUT ORIGINAL:")
        print(f"   {example['input_text'][:150]}...")
        print(f"\n🎯 TARGET ESPERADO:")
        print(f"   {example['target_text']}")

        # Generar respuesta del modelo
        inputs = tokenizer(
            example['input_text'],
            return_tensors="pt",
            truncation=True,
            max_length=256
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=128,
                num_beams=1,
                temperature=0.7,
                do_sample=True
            )

        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extraer solo la parte de respuesta
        input_decoded = tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)
        if generated.startswith(input_decoded):
            generated_response = generated[len(input_decoded):].strip()
        else:
            generated_response = generated

        print(f"\n🤖 RESPUESTA GENERADA:")
        print(f"   {generated_response}")

        # Calcular ROUGE manualmente para ver qué pasa
        from rouge_score import rouge_scorer

        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(example['target_text'], generated_response)

        print(f"\n📊 ROUGE manual:")
        print(f"   Rouge-1: {scores['rouge1'].fmeasure:.4f}")
        print(f"   Rouge-2: {scores['rouge2'].fmeasure:.4f}")
        print(f"   Rouge-L: {scores['rougeL'].fmeasure:.4f}")

        # Verificar si hay números en ambas respuestas
        import re
        target_numbers = set(re.findall(r'\d+\.?\d*', example['target_text']))
        generated_numbers = set(re.findall(r'\d+\.?\d*', generated_response))

        print(f"\n🔢 Números en target: {target_numbers}")
        print(f"🔢 Números en generado: {generated_numbers}")
        print(f"📐 Coincidencia numérica: {len(target_numbers.intersection(generated_numbers))}/{len(target_numbers)}")

        print("-" * 80)

# Ejecutar diagnóstico
debug_rouge_metrics(config.output_dir, dataset["test"], num_examples=6)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



🔍 DIAGNÓSTICO DE MÉTRICAS ROUGE
Analizando 6 ejemplos del test set...


--- Ejemplo 1 ---
📝 INPUT ORIGINAL:
   ### Instrucción:
¿Cómo evolucionó el porcentaje de voto a VOX en Medina del Campo entre Noviembre 2019 y Abril 2019?

### Contexto:
Municipio: Medina ...

🎯 TARGET ESPERADO:
   El VOX en Medina del Campo pasó de 19.86% en Noviembre 2019 a 13.75% en Abril 2019. Esto representa una disminución de 6.11 puntos porcentuales (30.8% menos).

🤖 RESPUESTA GENERADA:
   <pad> El VOX en Medina del Campo pasó de 37.53% en Noviembre 2019 a 33.79% en Abril 2019. Esto representa un incremento de 6.28% (de incremento porcentual). Esto representa un incremento de 1.64% (de incremento porcentual). Esto representa un incremento de 4.09 puntos porcentuales (0.25% disminución). Esto representa un incremento de 1.39 puntos porcentuales (0.66% disminución porcent

📊 ROUGE manual:
   Rouge-1: 0.4646
   Rouge-2: 0.3505
   Rouge-L: 0.4444

🔢 Números en target: {'19.86', '2019.', '13.75', '2019', '30.8'